In [37]:
import pickle
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [38]:
from sklearn.pipeline import make_pipeline

In [41]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("fakenews_rf")

<Experiment: artifact_location='s3://mlflow-artifacts-remote-rollan/1', creation_time=1693219091636, experiment_id='1', last_update_time=1693219091636, lifecycle_stage='active', name='fakenews_rf', tags={}>

In [39]:
import spacy

nlp = spacy.load("en_core_web_lg")

Read and download data

In [40]:
def read_data(filename: str) -> pd.DataFrame:
    """Read data into dataframe"""
    df = pd.read_csv(filename, index_col=0)
    df['vector'] = df['vector'].apply(convert)
    return df

def convert(item):
    item = item.strip()  # remove spaces at the end
    item = item[1:-1]    # remove `[ ]`
    item = np.fromstring(item, sep=' ')  # convert string to `numpy.array`
    return item

In [21]:
df_train = read_data("../../data/train_data.csv")
df_val = read_data("../../data/valid_data.csv")

In [27]:
def add_features(df_train, df_val):
    """"""
    X_train = df_train['vector']
    y_train = df_train['label']

    X_val = df_val['vector']
    y_val = df_val['label']

    X_train = np.stack(X_train)
    y_train = np.stack(y_train)

    X_val = np.stack(X_val)
    y_val = np.stack(y_val)
    return X_train, y_train, X_val, y_val

In [28]:
X_train, y_train, X_val, y_val = add_features(df_train, df_val)

In [ ]:
X_train

In [42]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    model = RandomForestRegressor(**params, n_jobs=-1)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(model, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 0.09719140266884883
